# Tutorial: A General-Purpose Photon Geodesic Integrator

## Author: Dalton Moone

## This notebook constructs a complete C-language project that numerically integrates the path of a photon in a specified spacetime. The core of the project is the numerical solution of the geodesic equation, which describes the path of a free-falling particle (or photon) through curved spacetime.

The geodesic equation, as detailed on [Wikipedia](https://en.wikipedia.org/wiki/Geodesic_equation), is a second-order ordinary differential equation (ODE) that relates the acceleration of a particle to the curvature of spacetime, represented by the Christoffel symbols ($\Gamma^{\alpha}_{\mu\nu}$):

$$ \frac{d^2x^{\alpha}}{d\lambda^2} = -\Gamma^{\\alpha}_{\mu\nu} \frac{dx^{\mu}}{d\lambda} \frac{dx^{\nu}}{d\lambda} $$

To solve this numerically, we convert this single second-order ODE into a system of two coupled first-order ODEs. This is a standard technique that makes the problem compatible with common ODE integrators. We achieve this by introducing a "dummy" variable called the **pseudo-momentum**, $p^{\alpha}$, defined as the velocity with respect to the affine parameter $\lambda$:

$$ p^{\alpha} \equiv \frac{dx^{\alpha}}{d\lambda} $$

This allows us to split the geodesic equation into the following system of 8 first-order ODEs (4 for position $x^{\alpha}$ and 4 for pseudo-momentum $p^{\alpha}$), which is what our C code will solve:

1.  **The Position ODE**:
    $ \frac{dx^{\alpha}}{d\lambda} = p^{\alpha} $
2.  **The Momentum ODE**:
    $ \frac{dp^{\alpha}}{d\lambda} = -\Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu} $

This notebook follows a modular, single-responsibility design pattern. It uses the `nrpy` library to first define the underlying physics symbolically, and then automatically generates a series of interoperable C functions, each with a specific job. This makes the final C project clear, efficient, and easily extensible to new spacetime metrics.

**Notebook Status:** <font color='green'><b>Validated</b></font>

# Table of Contents

This notebook is organized into a series of logical steps, with each core Python function encapsulated in its own cell. This modular design enhances readability and maintainability.

1.  [Step 1: Project Initialization](#initialize)
2.  [Step 2: The Symbolic Core - Foundational Math](#symbolic_core)
    1.  [2.a: Metric Tensor Derivatives](#deriv_g4DD)
    2.  [2.b: Christoffel Symbol Calculation](#four_connections)
    3.  [2.c: Geodesic Momentum RHS](#geodesic_mom_rhs)
    4.  [2.d: Geodesic Position RHS](#geodesic_pos_rhs)
    5.  [2.e: Symbolic Calculation of p⁰](#geodesic_mom0_calc)
3.  [Step 3: Spacetime and Symbolic Execution](#spacetime_and_execution)
    1.  [3.a: The Schwarzschild Metric](#schwarzschild_metric)
    2.  [3.b: Symbolic Workflow Execution](#symbolic_execution)
4.  [Step 4: C Code Generation - Physics "Engines"](#generate_c_engines)
    1.  [4.a: `g4DD_schwarzschild()` Worker](#g4DD_schwarzschild_engine)
    2.  [4.b: `con_schwarzschild()` Worker](#con_schwarzschild_engine)
    3.  [4.c: `calculate_p0()` Engine](#calculate_p0_engine)
    4.  [4.d: `calculate_ode_rhs()` Engine](#calculate_ode_rhs_engine)
5.  [Step 5: C Code Generation - Dispatchers and Orchestrators](#generate_c_orchestrators)
    1.  [5.a: `g4DD_metric()` Dispatcher](#g4DD_metric_dispatcher)
    2.  [5.b: `connections()` Dispatcher](#connections_dispatcher)
    3.  [5.c: `initial_data()` Orchestrator](#initial_data_orchestrator)
    4.  [5.d: The GSL Wrapper Function](#gsl_wrapper)
    5.  [5.e: The Main Integration Loop](#integration_loop)
    6.  [5.f: The `main()` C Function Entry Point](#main_entry_point)
6.  [Step 6: Project Assembly](#assemble_project)
    1.  [6.a: Custom Data Structures](#register_structs)
    2.  [6.b: Final Build](#final_build)

<a id='initialize'></a>
# Step 1: Project Initialization

This cell sets up the foundational elements for our entire project. It performs three key tasks:

1.  **Import Libraries**: We import necessary modules from standard Python libraries (`os`, `shutil`, `sympy`) and the core components of `nrpy`. The `nrpy` imports provide tools for C function registration (`nrpy.c_function`), C code generation (`nrpy.c_codegen`), parameter handling (`nrpy.params`), and infrastructure management (`nrpy.infrastructures.BHaH`).

2.  **Directory Management**: A clean output directory, `project/photon_geodesic_integrator/`, is created to store the generated C code, ensuring a fresh build every time the notebook is run.

3.  **Physical Parameter Definition**: We define the black hole mass, `M_scale`, as a `CodeParameter`. This powerful `nrpy` object, found in `nrpy/params.py`, registers the parameter with the BHaH infrastructure. We use several flags to control its behavior:
    *   `commondata=True`: This specifies that `M_scale` is a parameter that is "common" to the entire simulation, not specific to a single grid. As a result, it will be placed in the `commondata_struct` in the generated C code.
    *   `add_to_parfile=True`: This instructs the build system to add `M_scale` to a default parameter file (`.par`), making it easy to change at runtime. This is handled by `nrpy.infrastructures.BHaH.cmdline_input_and_parfiles`.
    *   `add_to_set_CodeParameters_h=True`: This is a crucial flag that enables the "automatic unpacking" mechanism. It tells `nrpy` to add an entry for `M_scale` to the `set_CodeParameters.h` convenience header. Any C function that includes this header will get a local `const REAL M_scale` variable, making the C code clean and readable. This is handled by `nrpy.infrastructures.BHaH.CodeParameters`.

In [1]:
import os
import shutil
import sympy as sp

import nrpy.c_function as cfc
import nrpy.c_codegen as ccg
import nrpy.params as par
import nrpy.indexedexp as ixp
import nrpy.infrastructures.BHaH.BHaH_defines_h as Bdefines_h
import nrpy.infrastructures.BHaH.Makefile_helpers as Makefile
from nrpy.infrastructures.BHaH import cmdline_input_and_parfiles
import nrpy.helpers.generic as gh


# Set project name and clean the output directory
project_name = "photon_geodesic_integrator"
project_dir = os.path.join("project", project_name)
shutil.rmtree(project_dir, ignore_errors=True)

# Set NRPy parameters for the BHaH infrastructure
par.set_parval_from_str("Infrastructure", "BHaH")

M_scale = par.CodeParameter(
    "REAL",
    __name__,
    "M_scale",
    1.0,
    add_to_parfile=True,
    commondata=True,
    add_to_set_CodeParameters_h=True
)

<a id='symbolic_core'></a>
# Step 2: The Symbolic Core - Foundational Math

This section defines the pure mathematical logic of our problem using Python's `sympy` library. Each function in this section is a "blueprint" for a physical calculation. These functions take symbolic `sympy` objects as input and return new symbolic expressions as output. They have no knowledge of C code; they are concerned only with mathematics and will be called later to generate the "recipes" for our C code engines.

<a id='deriv_g4DD'></a>
### 2.a: Metric Tensor Derivatives

The first step in calculating the Christoffel symbols is to compute the partial derivatives of the metric tensor, $g_{\mu\nu}$. This function, `derivative_g4DD`, takes the symbolic 4x4 metric tensor `g4DD` and a list of the four coordinate symbols `xx` as input.

It initializes a storage container for the results using a helper function from `nrpy`'s `indexedexp` module, which can be found in `nrpy/indexedexp.py`:
*   **`ixp.zerorank3(dimension=4)`**: This function creates a symbolic rank-3 tensor (a Python list of lists of lists) of a specified dimension, with all elements initialized to the `sympy` integer 0.

The function then iterates through all components to symbolically calculate the partial derivative of each metric component with respect to each coordinate. The resulting quantity, which we can denote using comma notation as $g_{\mu\nu,\alpha}$, is defined as:

$$ g_{\mu\nu,\alpha} \equiv \frac{\partial g_{\mu\nu}}{\partial x^{\alpha}} $$

The nested `for` loops in the code directly correspond to the spacetime indices `μ, ν, α` in the physics equation. `sympy`'s built-in `sp.diff()` function is used to perform the symbolic differentiation, and the final result is returned as a rank-3 symbolic tensor.

In [2]:
def derivative_g4DD(g4DD, xx):
    """Computes the symbolic first derivatives of the metric tensor."""
    g4DD_dD = ixp.zerorank3(dimension=4)
    for nu in range(4):
        for mu in range(4):
            for alpha in range(4):
                g4DD_dD[nu][mu][alpha] = sp.diff(g4DD[nu][mu], xx[alpha])
    return g4DD_dD

<a id='four_connections'></a>
### 2.b: Christoffel Symbol Calculation

This function implements the core formula for the Christoffel symbols of the second kind, $\Gamma^{\delta}_{\mu\nu}$. It takes the symbolic metric tensor `g4DD` ($g_{\mu\nu}$) and its derivatives `g4DD_dD` ($g_{\mu\nu,\alpha}$) as input. The calculation requires the inverse metric, $g^{\mu\nu}$, which is computed using another `nrpy` helper function from `nrpy/indexedexp.py`:

*   **`ixp.symm_matrix_inverter4x4(g4DD)`**: This function takes a symbolic 4x4 symmetric matrix and analytically computes its inverse. It is highly optimized for this specific task, returning both the inverse matrix and its determinant.

The function then applies the well-known formula, which can be found on [Wikipedia](https://en.wikipedia.org/wiki/Christoffel_symbols#Christoffel_symbols_of_the_second_kind). Using the comma notation for partial derivatives that we introduced in the previous step, the formula is:

$$ \Gamma^{\delta}_{\mu\nu} = \frac{1}{2} g^{\delta\alpha} \left( g_{\nu\alpha,\mu} + g_{\mu\alpha,\nu} - g_{\mu\nu,\alpha} \right) $$

The nested `for` loops in the Python code iterate over the spacetime indices `δ, μ, ν, α` to construct each component of the Christoffel symbol tensor. After each component is fully computed, `sympy`'s `sp.simplify()` function is used to reduce the resulting symbolic expression to its simplest form. The final rank-3 tensor of Christoffel symbols is then returned.

In [3]:
def four_connections(g4DD, g4DD_dD):
    """Computes and simplifies Christoffel symbols from the metric and its derivatives."""
    Gamma4UDD = ixp.zerorank3(dimension=4)
    g4UU, _ = ixp.symm_matrix_inverter4x4(g4DD)
    for mu in range(4):
        for nu in range(4):
            for delta in range(4):
                for alpha in range(4):
                    Gamma4UDD[delta][mu][nu] += sp.Rational(1, 2) * g4UU[delta][alpha] * (g4DD_dD[nu][alpha][mu] + g4DD_dD[mu][alpha][nu] - g4DD_dD[mu][nu][alpha])
                Gamma4UDD[delta][mu][nu] = sp.simplify(Gamma4UDD[delta][mu][nu])
    return Gamma4UDD

<a id='geodesic_mom_rhs'></a>
### 2.c: Geodesic Momentum RHS

This function defines the symbolic right-hand side (RHS) for the evolution of the **pseudo-momentum**, $p^{\alpha}$. As established, this is the second of our two first-order ODE systems:
$$ \frac{dp^{\alpha}}{d\lambda} = -\Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu} $$
where $p^{\mu}$ is the pseudo-momentum and $\lambda$ is the affine parameter.

The function `geodesic_mom_rhs` takes the symbolic Christoffel symbols $\Gamma^{\alpha}_{\mu\nu}$ as its input. It then defines the symbolic pseudo-momentum vector `pU` using `sympy`'s `sp.symbols()` function. Notice that we use "direct naming" here: the symbols are created with names that are already valid C array syntax (e.g., `y[4]`). This is a key `nrpy` technique that simplifies the final code generation by eliminating the need for string substitutions.

The core of this function constructs the symbolic expression for the RHS. A direct implementation of the Einstein summation notation $-\Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu}$ would involve a double loop over both $\mu$ and $\nu$ from 0 to 3. This would result in $4 \times 4 = 16$ terms for each of the four components of $\alpha$, which is computationally inefficient.

However, we can significantly optimize this calculation by exploiting symmetry. The term $p^{\mu} p^{\nu}$ is symmetric with respect to the interchange of the indices $\mu$ and $\nu$. The Christoffel symbols $\Gamma^{\alpha}_{\mu\nu}$ are also symmetric in their lower two indices. Therefore, the full sum can be split into diagonal ($\mu=\nu$) and off-diagonal ($\mu \neq \nu$) terms:
$$ -\Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu} = -\sum_{\mu=0}^{3} \Gamma^{\alpha}_{\mu\mu} (p^{\mu})^2 - \sum_{\mu \neq \nu} \Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu} $$
The second sum over $\mu \neq \nu$ contains pairs of identical terms. For example, the term for $(\mu=1, \nu=2)$ is $\Gamma^{\alpha}_{12} p^{1} p^{2}$, and the term for $(\mu=2, \nu=1)$ is $\Gamma^{\alpha}_{21} p^{2} p^{1}$. Since $\Gamma^{\alpha}_{12} = \Gamma^{\alpha}_{21}$, these terms are the same. We can combine all such pairs by summing over only one of the cases (e.g., $\mu < \nu$) and multiplying by two:
$$ -\Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu} = -\sum_{\mu=0}^{3} \Gamma^{\alpha}_{\mu\mu} (p^{\mu})^2 - 2 \sum_{\mu < \nu} \Gamma^{\alpha}_{\mu\nu} p^{\mu} p^{\nu} $$
The Python code implements this optimized version. The outermost loop, `for alpha in range(4)`, iterates through each of the four components of the RHS vector we are constructing ($dp^0/d\lambda, dp^1/d\lambda, \dots$). Inside this, the loop `for mu in range(4)` handles the diagonal terms ($\Gamma^{\alpha}_{\mu\mu}(p^{\mu})^2$), and the innermost loop `for nu in range(mu + 1, 4)` handles the unique off-diagonal terms (where `nu > mu`), correctly multiplying them by 2. This structure ensures that for each component $\alpha$, the sum is computed with the minimum number of floating point operations, leading to more efficient C code.

In [4]:
def geodesic_mom_rhs(Gamma4UDD):
    pt,pr,pth,pph = sp.symbols("y[4] y[5] y[6] y[7]", Real = True)
    pU =[pt,pr,pth,pph]
    geodesic_rhs = ixp.zerorank1(dimension=4)
    for alpha in range(4):
        for mu in range(4):
            geodesic_rhs[alpha] += -Gamma4UDD[alpha][mu][mu] * pU[mu] * pU[mu]
            for nu in range(mu + 1, 4):
                geodesic_rhs[alpha] += -2 * Gamma4UDD[alpha][mu][nu] * pU[mu] * pU[nu]
    return geodesic_rhs

<a id='geodesic_pos_rhs'></a>
### 2.d: Geodesic Position RHS

This function defines the symbolic right-hand side (RHS) for the evolution of the position coordinates, $x^{\alpha}$. This is the first of our two first-order ODE systems. It follows directly from the definition of the pseudo-momentum, $p^{\alpha}$, that we introduced to convert the single second-order geodesic equation into a system of first-order ODEs:

$$ \frac{dx^{\alpha}}{d\lambda} = p^{\alpha} $$

The Python function `geodesic_pos_rhs` is straightforward. It defines the components of the pseudo-momentum vector, `pU`, using `sympy`'s `sp.symbols()` function with the "direct naming" convention (`y[4]`, `y[5]`, etc.). It then simply returns this vector.

This list of four symbolic expressions will serve as the first four components of the complete 8-component RHS vector that our C code will solve.

In [5]:
def geodesic_pos_rhs():
    pt,pr,pth,pph = sp.symbols("y[4] y[5] y[6] y[7]", Real = True)
    pU =[pt,pr,pth,pph]

    return pU

<a id='geodesic_mom0_calc'></a>
### 2.e: Symbolic Calculation of p⁰

To complete our initial data, we must enforce the **null geodesic condition**, which states that the squared 4-momentum of a photon is zero. This is because photons travel along null paths where the spacetime interval $ds^2$ is zero. This condition is expressed as:

$$ g_{\mu\nu} p^{\mu} p^{\nu} = 0 $$

Our goal is to solve this equation for the time-component of the pseudo-momentum, $p^0$. To do this, we must first expand the Einstein summation over the 4D spacetime indices (μ, ν = 0, 1, 2, 3) into separate time (`0`) and space (`i, j = 1,2,3`) components. The expansion yields:
$$ g_{00}(p^0)^2 + g_{0i}p^0 p^i + g_{i0}p^i p^0 + g_{ij}p^i p^j = 0 $$

Since the metric is symmetric ($g_{i0} = g_{0i}$), we can combine the middle terms:

$$ g_{00}(p^0)^2 + 2(g_{0i}p^i)p^0 + (g_{ij}p^i p^j) = 0 $$

This is a standard quadratic equation of the form $ax^2 + bx + c = 0$, where $x = p^0$. The coefficients are:
*   $a = g_{00}$
*   $b = 2 \sum_{i=1}^3 g_{0i}p^i$
*   $c = \sum_{i=1}^3 \sum_{j=1}^3 g_{ij}p^i p^j$

The solution for $p^0$ is given by the [quadratic formula](https://en.wikipedia.org/wiki/Quadratic_formula):

$$ p^0 = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} = \frac{1}{g_{00}} \left[ -g_{0i}p^i \pm \sqrt{(g_{0i}p^i)^2 - g_{00}(g_{ij}p^i p^j)} \right] $$

We must choose the sign that results in a positive $p^0$, which corresponds to a **future-directed** photon (one moving forward in time). The function `mom_time()` below constructs the symbolic `sympy` expression for this formula. It uses a helper function from the `nrpy.indexedexp` module to create abstract symbolic tensors:

*   **`ixp.declarerank2("g", sym="sym01", dimension=4)`**: This function, found in `nrpy/indexedexp.py`, creates a symbolic rank-2 tensor (a 4x4 matrix) named `g`. The `sym="sym01"` argument automatically enforces the symmetry condition $g_{\mu\nu} = g_{\nu\mu}$, which is essential for our derivation.

The function then programmatically builds the summation terms and combines them according to the quadratic formula, returning a single, complex `sympy` expression. This expression is the "recipe" that our C code generator will use later.

In [6]:
def mom_time():
    p0,p1,p2,p3 = sp.symbols("y[4] y[5] y[6] y[7]", Real = True)
    pU=[p0,p1,p2,p3]
    g4DD = ixp.declarerank2("g", sym= "sym01",dimension =4)

    sum_g0i_pi =sp.sympify(0)
    for i in range(1,4):
        sum_g0i_pi += g4DD[0][i]*pU[i]


    sum_gij_pi_pj= sp.sympify(0)

    for i in range(1,4):
        sum_gij_pi_pj += g4DD[i][i]*pU[i]*pU[i]
        for j in range(i+1,4):
            sum_gij_pi_pj += 2*g4DD[i][j]*pU[i]*pU[j]


    answer = -sum_g0i_pi - sp.sqrt(sum_g0i_pi*sum_g0i_pi-g4DD[0][0]*sum_gij_pi_pj)
    # took negative value for sqrt to make p^0>0 since g00 is likly to negative
    return answer/g4DD[0][0]




<a id='spacetime_and_execution'></a>
# Step 3: Spacetime Definition and Symbolic Workflow

This section marks the transition from defining abstract mathematical tools to applying them. We will now perform two key tasks:

1.  **Define a Specific Spacetime:** We will implement the Schwarzschild metric, which describes the geometry around a non-rotating, uncharged black hole. This provides the specific `g_munu` that our abstract functions will operate on.
2.  **Execute the Symbolic Workflow:** We will call the symbolic "blueprint" functions defined in Step 2 in the correct sequence. This will generate the final, complex `sympy` expressions for the Christoffel symbols and the right-hand-sides of the geodesic equations. These symbolic expressions are then stored in global Python variables, ready to be used by the C code generators in the subsequent steps.

<a id='schwarzschild_metric'></a>
### 3.a: The Schwarzschild Metric

This function defines the [Schwarzschild metric](https://en.wikipedia.org/wiki/Schwarzschild_metric) in spherical coordinates, which describes the spacetime geometry around a spherically symmetric, non-rotating, uncharged massive object.

The function first defines the symbolic coordinates for this spacetime ($t, r, \theta, \phi$) using `sympy`'s `sp.symbols()` function with the "direct naming" convention (`y[0], y[1], ...`). It then constructs the 4x4 metric tensor, $g_{\mu\nu}$, as a symbolic `sympy` matrix.

The non-zero components of the metric are:
$$ g_{tt} = -\left(1 - \frac{2M}{r}\right) \quad , \quad g_{rr} = \left(1 - \frac{2M}{r}\right)^{-1} \quad , \quad g_{\theta\theta} = r^2 \quad , \quad g_{\phi\phi} = r^2 \sin^2\theta $$

These components are constructed symbolically using the abstract `M_scale.symbol` to represent the mass of the black hole. The function returns the symbolic rank-2 tensor `g4DD` ($g_{\mu\nu}$) and the Python list of coordinate symbols, which will be used in the next step to compute derivatives.

In [7]:
def define_schwarzschild_metric_analytic():
    """Defines and returns the ANALYTIC Schwarzschild metric tensor and its coordinate system."""
    t, r, th, ph = sp.symbols("y[0] y[1] y[2] y[3]", real=True)
    xx = [t, r, th, ph]
    g4DD = ixp.zerorank2(dimension=4)
    
    M_sym = M_scale.symbol
    _1 = sp.sympify(1)
    _2 = sp.sympify(2)
    
    g4DD[0][0] = -(_1 - _2*M_sym/r)
    g4DD[1][1] = _1 / (_1 - _2*M_sym/r)
    g4DD[2][2] = r**2
    g4DD[3][3] = r**2 * sp.sin(th)**2
    return g4DD, xx


<a id='symbolic_execution'></a>
## 3.b: Symbolic Workflow Execution

This cell acts as the central hub for the symbolic portion of our project. In the preceding cells, we *defined* a series of Python functions that perform individual mathematical tasks. Here, we *execute* those functions in the correct sequence to generate all the final symbolic expressions that will serve as "recipes" for our C code generators.

This "symbolic-first" approach is a core `nrpy` principle and offers significant advantages:
1.  **Efficiency**: The complex symbolic calculations, such as inverting the metric tensor and deriving the Christoffel symbols, are performed **only once** when this notebook is run. The results are stored in global Python variables, preventing redundant and time-consuming recalculations in the C-generating cells.
2.  **Modularity**: This workflow creates a clean separation between the *specific solution* for a metric (the explicit formulas for the Schwarzschild Christoffels) and the *generic form* of the equations of motion (which are valid for any metric).

This cell produces two distinct sets of symbolic expressions that are stored in global variables for later use:

*   **`Gamma4UDD`**: This rank-3 tensor holds the **explicit symbolic formulas** for the Christoffel symbols ($\Gamma^{\alpha}_{\mu\nu}$) of the Schwarzschild metric. It is generated by calling `define_schwarzschild_metric_analytic()`, `derivative_g4DD()`, and `four_connections()`. This variable will be used exclusively by the `con_schwarzschild()` C code generator.

*   **`all_rhs_expressions`**: This is a Python list containing the 8 symbolic expressions for the right-hand-sides of our ODE system. To keep this part of the code generic and independent of any specific metric, we create a symbolic **placeholder** for the Christoffel symbols using `ixp.declarerank3("conn->Gamma4UDD", ...)`. This placeholder is passed to `geodesic_mom_rhs()` to construct the geodesic equation in its abstract form. This elegant technique embeds the final C variable name (`conn->Gamma4UDD...`) directly into the symbolic expression, which dramatically simplifies the C code generation step for the `calculate_ode_rhs()` engine.

In [8]:
# --- 1. Define the metric and get the coordinate symbols ---
# This creates the global variables `g4DD` (metric) and `xx` (coordinates).
g4DD, xx = define_schwarzschild_metric_analytic()
# We can also unpack xx into individual global variables for convenience.
t, r, th, ph = xx
print(" -> Defined global symbolic variables: g4DD, xx, t, r, th, ph")

# --- 2. Calculate the Christoffel symbols ---
# This creates the global variable `Gamma4UDD` containing the symbolic formulas.
g4DD_dD = derivative_g4DD(g4DD, xx)
Gamma4UDD = four_connections(g4DD, g4DD_dD)
print(" -> Defined global symbolic variable: Gamma4UDD")

# --- 3. Define the symbolic pseudo-momentum vector ---
# This creates the global variable `pU`.
print(" -> Defined global symbolic variable: pU")

# --- 4. Generate the symbolic RHS expressions for the geodesic equations ---
# Create a GENERIC placeholder for the Christoffel symbols
Gamma4UDD_placeholder = ixp.declarerank3("conn->Gamma4UDD", dimension=4)
rhs_pos = geodesic_pos_rhs()
rhs_mom = geodesic_mom_rhs(Gamma4UDD_placeholder)
all_rhs_expressions = rhs_pos + rhs_mom
print(" -> Defined global symbolic variable: all_rhs_expressions")
print("\nSymbolic setup complete. All expressions are now available globally.")

 -> Defined global symbolic variables: g4DD, xx, t, r, th, ph
 -> Defined global symbolic variable: Gamma4UDD
 -> Defined global symbolic variable: pU
 -> Defined global symbolic variable: all_rhs_expressions

Symbolic setup complete. All expressions are now available globally.


<a id='generate_c_engines'></a>
# Step 4: C Code Generation - Physics "Engines"

This section marks our transition from pure symbolic mathematics to C code generation. The Python functions defined here are "meta-functions": their job is not to perform calculations themselves, but to **generate the C code** that will perform the calculations in the final compiled program. These generated C functions are our low-level "physics engines," each responsible for a single, specific task.

<a id='g4DD_schwarzschild_engine'></a>
### 4.a: `g4DD_schwarzschild()` Worker

This Python function, `g4DD_schwarzschild()`, generates its C-language counterpart. This C function is a specialized **worker** whose only job is to compute the 10 unique components of the Schwarzschild metric tensor, $g_{\mu\nu}$, at a given point in spacetime.

The generation process is as follows:
1.  **Access Symbolic Recipe:** It calls our `define_schwarzschild_metric_analytic()` function to get the symbolic `sympy` expression for the metric tensor.
2.  **Define C Assignment:** It creates two Python lists: one containing the 10 symbolic metric expressions (the right-hand-sides) and another containing the corresponding C variable names for the members of the `metric_struct` (e.g., `metric->g00`, the left-hand-sides).
3.  **Generate C Code:** It passes these two lists to `nrpy.c_codegen.c_codegen` (from `nrpy/c_codegen.py`). This powerful `nrpy` function converts the symbolic math into highly optimized C code, including performing Common Subexpression Elimination (CSE) to reduce redundant calculations.
4.  **Register C Function:** Finally, it bundles the generated C code `body` with its metadata (like its function signature) and registers the complete function with `nrpy.c_function.register_CFunction`. Crucially, it sets `include_CodeParameters_h=True`, which automatically handles access to the `M_scale` parameter.

In [9]:
def con_schwarzschild():
# --- 1. Generate the metric-specific connection function ---
    list_of_connections = []
    conn_Gamma4UDD = ixp.declarerank3("conn->Gamma4UDD", dimension=4)
    for i in range(4):
        for j in range(4):
            for k in range(j,4):
                list_of_connections.append(str(conn_Gamma4UDD[i][j][k]))


    Gamma=[]
    for i in range(4):
        for j in range(4):
            for k in range(j,4):
                Gamma.append(Gamma4UDD[i][j][k])





    ########################################################################
    includes=["BHaH_defines.h"]
    desc = r"""@brief Computes the 40 unique Christoffel symbols for the Schwarzschild metric.
        
        This function is a specialized "worker" that calculates the Christoffel
        symbols at a single point in spacetime for the Schwarzschild metric. It is
        called by the connections() dispatcher.
        
        The body of this function is entirely auto-generated by NRPy's c_codegen
        module from the symbolic formulas derived in Python. This process includes
        Common Subexpression Elimination (CSE) to create temporary variables
        (e.g., tmp0, tmp2), which reduces the number of floating-point operations
        and improves the function's performance. The function explicitly calculates
        and assigns all 40 unique Christoffel symbols to the members of the output struct.
        
        @param[in]  params A pointer to the params_struct containing all physical parameters (e.g., M_scale).
        @param[in]  y      The state vector, from which the first 4 elements are used as the coordinates (t, r, th, ph).
        @param[out] conn   A pointer to the connection_struct that will be filled with the results."""
    name="con_schwarzschild"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const double y[4], connection_struct *restrict conn"


    body=ccg.c_codegen(
        Gamma,
        list_of_connections,
        enable_cse=True,

        )


    cfc.register_CFunction(
        includes=includes,
        desc= desc,
        name=name,
        params= params,
        body=body,
        include_CodeParameters_h=True
    )


<a id='con_schwarzschild_engine'></a>
### 4.b: `con_schwarzschild()` Worker

This function is structured identically to the `g4DD_schwarzschild` worker. It generates the C **worker** function `con_schwarzschild()`, whose only job is to compute the 40 unique Christoffel symbols, $\Gamma^{\alpha}_{\mu\nu}$, for the Schwarzschild metric.

It accesses the pre-computed symbolic Christoffel formulas from the global `Gamma4UDD` variable (which was created in Step 3.b). It then uses `nrpy.c_codegen.c_codegen` to convert these `sympy` expressions into optimized C code that populates the members of the `connection_struct`. Like the metric worker, it uses the `include_CodeParameters_h=True` flag to automatically handle the dependency on `M_scale`, which is needed to calculate the Christoffel symbols.

In [10]:
def connections():
    print(" -> Generating C dispatcher: connections()...")


    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "stdio.h", "stdlib.h"]
    desc = r"""@brief Dispatcher to compute Christoffel symbols for the chosen metric.
    
    This function acts as a high-level "manager" that selects the correct
    metric-specific "worker" function to calculate the Christoffel symbols.
    
    It uses a C switch statement to evaluate the 'type' member of the
    metric_params struct. Based on this value, it dispatches the call to the
    appropriate specialized function (e.g., con_schwarzschild()). This design
    pattern makes the code highly extensible, as adding new spacetime metrics
    simply requires adding a new case to the switch statement.
    
    @param[in]  params    A pointer to the params_struct containing all physical parameters.
    @param[in]  metric    A pointer to the metric_params struct, which specifies the metric to use.
    @param[in]  y         The 8-component state vector, used to pass coordinates to the worker function.
    @param[out] conn      A pointer to the connection_struct that will be filled by the worker function."""
    
    name = "connections"
    cfunc_type = "void" 


    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const metric_params *restrict metric, const double y[8], connection_struct *restrict conn"


    body = r"""
    // This switch statement chooses which "worker" function to call
    // based on the metric type provided.
    switch(metric->type) {
        case Schwarzschild:
        // For Schwarzschild, call the specialist function.
        // It needs the main parameters (for M_scale) and the coordinates (y).
        con_schwarzschild(commondata, params, y, conn);
        break;
        // Placeholders for future expansion.
        case Kerr:
        case Numerical:
        default:
        printf("Error: MetricType %d not supported yet.\n", metric->type);
        exit(1);
        break;
    }
    """


    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )
    print("    ... connections() registration complete.")

<a id='g4DD_schwarzschild_engine'></a>
### 4.c: `g4DD_schwarzschild()` Worker

This Python function, `g4DD_schwarzschild()`, generates its C-language counterpart. This C function is a specialized **worker** whose only job is to compute the 10 unique components of the Schwarzschild metric tensor, $g_{\mu\nu}$, at a given point in spacetime.

The generation process is as follows:
1.  **Access Symbolic Recipe:** It calls our `define_schwarzschild_metric_analytic()` function to get the symbolic `sympy` expression for the metric tensor.
2.  **Define C Assignment:** It creates two Python lists: one containing the 10 symbolic metric expressions (the right-hand-sides) and another containing the corresponding C variable names for the members of the `metric_struct` (e.g., `metric->g00`, the left-hand-sides).
3.  **Generate C Code:** It passes these two lists to `nrpy.c_codegen.c_codegen` (from `nrpy/c_codegen.py`). This powerful `nrpy` function converts the symbolic math into highly optimized C code, including performing Common Subexpression Elimination (CSE) to reduce redundant calculations.
4.  **Register C Function:** Finally, it bundles the generated C code `body` with its metadata (like its function signature) and registers the complete function with `nrpy.c_function.register_CFunction`. Crucially, it sets `include_CodeParameters_h=True`, which automatically handles access to the `M_scale` parameter.

In [11]:
def g4DD_schwarzschild():
    """
    Generates and registers the C function to compute the Schwarzschild
    metric components, g_munu. This is a metric-specific "worker" function.
    """
    print(" -> Generating C worker function: g4DD_schwarzschild()...")
    
    # --- 1. The Symbolic Part ---
    # Call the existing function to get the symbolic expression for the metric.
    # We pass it the symbolic coordinate vector y_sym, which should be defined in a previous cell.
    g4DD, _ = define_schwarzschild_metric_analytic()
    # Create a list of the 10 unique symbolic expressions for the metric components.
    list_of_g4DD_syms = []
    for i in range(4):
        for j in range(i, 4):
            list_of_g4DD_syms.append(g4DD[i][j])

    # --- 2. The C-Code Generation Part ---
    # Create a list of the 10 C variable names for the metric struct members.
    list_of_g4DD_C_vars = []
    for i in range(4):
        for j in range(i, 4):
            list_of_g4DD_C_vars.append(f"metric->g{i}{j}")

    # Define the C function's signature.
    # It needs commondata and params for M_scale, y for coordinates, and a pointer to the output struct.
    includes = ["BHaH_defines.h"]
    desc = r"""@brief Computes the 10 unique components of the Schwarzschild metric.
    @param[in]  commondata  Pointer to commondata_struct.
    @param[in]  params      Pointer to params_struct.
    @param[in]  y           The state vector, containing coordinate information.
    @param[out] metric_out  Pointer to the metric_struct to be filled."""
    name = "g4DD_schwarzschild"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const double y[4], metric_struct *restrict metric"
   
    # Generate the C code body using CSE.
    # Note that the abstract M_scale.symbol will be correctly handled because we use include_CodeParameters_h=True below.
    body = ccg.c_codegen(list_of_g4DD_syms, list_of_g4DD_C_vars, enable_cse=True)
    # --- 3. The Registration Part ---
    # Register the C function with nrpy, including the flag to handle parameters.
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body,
        include_CodeParameters_h=True  # This is the key to making M_scale work.
    )
    print("    ... g4DD_schwarzschild() registration complete.")


<a id='g4DD_metric_dispatcher'></a>
### 4.d: `g4DD_metric()` Dispatcher

This Python function generates the C function `g4DD_metric()`, which acts as a high-level "manager" or **dispatcher.** Its sole responsibility is to select and call the correct metric-specific worker function (like `g4DD_schwarzschild()`) based on the `MetricType` enum.

The generated C code uses a `switch` statement based on the `metric->type` member of the `metric_params` struct. This design is highly extensible: to add support for a new metric (like Kerr), one would simply write a new worker function (`g4DD_kerr()`) and add a new `case` to this `switch` statement.

The function's signature and body are handwritten as C code strings, demonstrating how `nrpy` allows for the seamless integration of developer-written control flow with the automatically generated worker functions. This function does **not** need the `include_CodeParameters_h` flag, as it only passes pointers down the call chain and does not use any physical parameters directly.

In [12]:
def g4DD_metric():
    """
    Generates and registers the C function g4DD_metric(), which serves as a
    dispatcher to call the appropriate metric-specific worker function.
    """
    print(" -> Generating C dispatcher function: g4DD_metric()...")
    
    # Define the C function's signature.
    # It needs the parameters to pass down, the metric type to switch on,
    # the coordinates, and a pointer to the output struct.
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h"]
    desc = r"""@brief Dispatcher to compute the 4-metric g_munu for the chosen metric.
    @param[in]  commondata  Pointer to commondata_struct.
    @param[in]  params      Pointer to params_struct.
    @param[in]  metric      Pointer to the metric_params struct, which specifies the metric to use.
    @param[in]  y           The state vector, containing coordinate information.
    @param[out] metric_out  Pointer to the metric_struct to be filled by the worker function."""
    name = "g4DD_metric"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const metric_params *restrict metric, const double y[4], metric_struct *restrict metric_out"
    
    # The body is a handwritten C switch statement.
    # It calls the worker function you just created in the previous cell.
    body = r"""
    // This switch statement chooses which "worker" function to call
    // based on the metric type provided.
    switch(metric->type) {
        case Schwarzschild:
            // For Schwarzschild, call the specialist function.
            g4DD_schwarzschild(commondata, params, y, metric_out);
            break;
        // To add a new metric (e.g., Kerr), you would add a new case here:
        // case Kerr:
        //     g4DD_kerr(commondata, params, y, metric_out);
        //     break;
        default:
            printf("Error: MetricType %d not supported in g4DD_metric() yet.\\n", metric->type);
            exit(1);
            break;
    }
"""
    
    # Register the C function.
    # This dispatcher does NOT need include_CodeParameters_h, as it doesn't
    # use any parameters directly; it only passes pointers.
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body
    )
    print("    ... g4DD_metric() registration complete.")

<a id='calculate_p0_engine'></a>
### 4.c: `calculate_p0()` Engine

This Python function generates the C engine `calculate_p0()`, which implements the general formula for the time-component of the pseudo-momentum, $p^0$, derived in Step 2.e. This function is a prime example of a reusable component, as it is valid for any metric for which the components $g_{\mu\nu}$ are known.

The code generation follows a pattern that is both robust and highly automated, showcasing a powerful `nrpy` technique:

1.  **Symbolic Recipe:** It calls our pure-math `mom_time()` function to get the complete symbolic expression for $p^0$. This expression is built from abstract `sympy` symbols (e.g., `g00`, `g01`, etc.).
2.  **Preamble Generation:** The function then programmatically generates a C code "preamble." This preamble consists of a series of `const double` declarations that unpack the numerical values from the input `metric_struct` pointer and assign them to local C variables that have the *exact same names* as our abstract `sympy` symbols (e.g., `const double g00 = metric->g00;`).
3.  **C Code Generation:** It calls `nrpy.c_codegen.c_codegen` to convert the symbolic `p0_expr` into an optimized C expression, assigning it to a temporary variable `p0_val`. This works seamlessly because the symbols in the expression (`g00`, etc.) now match the local C variables created by the preamble.
4.  **Return Value:** The final C function body is constructed by combining the preamble, the CSE-optimized calculation, and a `return p0_val;` statement. This creates a complete, efficient, and readable C function without any manual C-string manipulation.

In [13]:
def calculate_p0():
    """
    Generates and registers the C function calculate_p0(), which computes
    the time component of the 4-momentum, p^0, from the null geodesic condition.
    """
    print(" -> Generating C engine function: calculate_p0()...")
    
    # --- 1. The Symbolic Part ---
    # Call our existing function to get the full symbolic expression for p^0.
    p0_expr = mom_time()
    
  
    # --- 2. The C-Code Generation Part ---
    # Define the C function's signature.
    # It takes the metric struct and the y state vector as input and returns a double.
    includes = ["BHaH_defines.h", "math.h"]
    desc = r"""@brief Computes p^0 from the null geodesic condition g_munu p^mu p^nu = 0.
    @param[in]  metric  Pointer to the metric_struct containing g_munu components.
    @param[in]  y       The state vector, containing the spatial momentum p^i.
    @return The positive, future-directed value for p^0."""
    name = "calculate_p0"
    c_type = "double"
    params = "const metric_struct *restrict metric, const double y[8]"

    # We need to map the abstract symbolic metric 'g' to the C struct 'metric'.
    # NRPy's c_codegen will see 'g01' and look for a C variable with that name.
    # We need to tell it that 'g01' is actually 'metric->g01'.
    # We can do this with a systematic list of substitutions.

    preamble = ""
    for i in range(4):
        for j in range(i, 4):
            preamble += f"const double g{i}{j} = metric->g{i}{j};\n"


    p0_C_code_lines = ccg.c_codegen(
        p0_expr, 
        'double p0_val', 
        enable_cse=True, 
        include_braces=False  
    )

    body = f"""{{
    {preamble}
    {p0_C_code_lines}
    return p0_val;
    }}"""

    # --- 3. The Registration Part ---
    # Register the C function.
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=c_type,
        name=name,
        params=params,
        body=body
    )
    print("    ... calculate_p0() registration complete.")

<a id='calculate_ode_rhs_engine'></a>
### 4.e: `calculate_ode_rhs()` Engine

This function generates the core "engine" of our ODE solver: the C function `calculate_ode_rhs()`. Its single responsibility is to calculate the right-hand sides for our entire system of 8 ODEs. It is completely generic and has no knowledge of any specific metric or its physical parameters (like `M_scale`); it only knows how to compute the geodesic equations given a set of Christoffel symbols.

The generation process is straightforward:
1.  **Access Generic Recipe:** It accesses the global `all_rhs_expressions` list, which contains the generic symbolic form of the geodesic equations.
2.  **Generate C Code:** It passes this list directly to `nrpy.c_codegen.c_codegen`. The symbols used to build `all_rhs_expressions` were already created with their final C syntax (e.g., `y[5]` for the pseudo-momentum and `conn->Gamma4UDD012` for the Christoffel placeholder). Therefore, no further symbolic manipulation or substitution is needed. `nrpy` simply translates the expressions into optimized C code.
3.  **Register C Function:** The generated C code body is bundled with its metadata and registered. This function does not require the `include_CodeParameters_h` flag because it is physically generic.

In [14]:
def calculate_ode_rhs():

    rhs_output_vars = [f"rhs_out[{i}]" for i in range(8)]



    includes = ["BHaH_defines.h"]

    desc = r"""@brief Calculates the right-hand sides (RHS) of the 8 geodesic ODEs.
 
    This function implements the generic geodesic equation using pre-computed
    Christoffel symbols. It is a pure "engine" function that does not depend
    on any specific metric's parameters (like M_scale), only on the geometric
    values passed to it via the connection struct.

    @param[in]  y         The 8-component state vector [t, r, th, ph, p^t, p^r, p^th, p^ph].
    @param[in]  conn      A pointer to the connection_struct holding the pre-computed Christoffel symbols.
    @param[out] rhs_out   A pointer to the 8-component output array where the RHS results are stored."""
            
    name = "calculate_ode_rhs"
    params = "const double y[8], const connection_struct *restrict conn, double rhs_out[8]"

    body=ccg.c_codegen(all_rhs_expressions,rhs_output_vars)

    cfc.register_CFunction(
        includes= includes,
        name=name,
        desc=desc,
        params=params,
        body=body
    )


<a id='initial_data_orchestrator'></a>
# Step 5: C Code Generation - Dispatchers and Orchestrators

With the low-level "engine" and "worker" functions defined in the previous step, we now generate the higher-level C functions that manage the simulation. These functions are responsible for dispatching to the correct worker based on runtime parameters and for orchestrating the overall program flow.

### 5.a: `initial_data()` Orchestrator

This Python function generates the C function `initial_data()`, which serves as a high-level **orchestrator** for setting the complete initial state of the photon. Instead of containing hard-coded physics, its only job is to call the low-level engines in the correct sequence.

The C function generated by this cell performs the following steps:
1.  **Set Independent Data:** It first sets the "free parameters" of the initial state, such as the starting radius (`y[1] = 4.0 * M_scale`) and the initial direction of the spatial momentum (`y[5]`, `y[6]`, `y[7]`). It can use `M_scale` here because it uses the `include_CodeParameters_h` flag.
2.  **Call Metric Dispatcher:** It declares a local `metric_struct` to hold the metric components and then calls our `g4DD_metric()` dispatcher. This populates the struct with the appropriate metric values (e.g., Schwarzschild) at the starting location.
3.  **Call p⁰ Engine:** With the metric and spatial momentum now known, it calls our `calculate_p0()` engine to compute the final, dependent component of the initial state vector, `y[4]`.

This modular design makes the code robust and readable. The `initial_data` function defines the "recipe" for initialization, while the underlying engines handle the detailed calculations.

In [15]:
def initial_data():
    """
    Generates the C function initial_data(), which sets the complete initial
    state vector y[8] for a photon, assuming a spherical-like coordinate system.
    """
    print(" -> Generating C function: initial_data()...")

    # Define the C function's signature.
    # It needs parameters for M_scale, the metric type, and the output array.
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "math.h"]
    desc = r"""@brief Sets the complete 8-component initial state vector y[8].
    This function sets the initial spatial position and momentum, then calls
    the necessary engines to compute g_munu and the dependent component p^0.
    @param[in]  commondata Pointer to commondata_struct for M_scale.
    @param[in]  params     Pointer to params_struct (required by set_CodeParameters.h).
    @param[in]  metric     Pointer to metric_params for metric type.
    @param[out] y_out      The 8-component state vector to be filled."""
    name = "initial_data"
    params = "const commondata_struct *restrict commondata, const params_struct *restrict params, const metric_params *restrict metric, double y_out[8]"
    
    # The body is a handwritten C function that orchestrates the initialization.
    body = r"""
    #ifndef M_PI
    #define M_PI 3.14159265358979323846
    #endif



    // Step 1: Set the independent components of the initial state vector.
    // This defines the initial state of the photon in our chosen coordinate system.
    // Set initial position (t, r, theta, phi)
    y_out[0] = 0.0;
    y_out[1] = 4.0 * M_scale; // Start at 4M
    y_out[2] = M_PI / 2.0;    // Start in the equatorial plane
    y_out[3] = 0.0;           // Start at phi = 0

    // Set initial SPATIAL momentum (p^r, p^theta, p^phi)
    y_out[5] = -1.0; // In-going radial momentum
    y_out[6] = 0.0;  // No initial theta momentum
    y_out[7] = 0.0;  // No initial phi momentum

    // Step 2: Declare a temporary struct to hold the metric components.
    metric_struct g4DD;

    // Step 3: Call the g4DD_metric dispatcher to compute g_munu for the chosen
    // metric at the starting position y_out.
    g4DD_metric(commondata, params, metric, y_out, &g4DD);

    // Step 4: Call the calculate_p0 engine to compute the dependent component p^0.
    // This engine uses the metric values and spatial momentum we just set.
    y_out[4] = calculate_p0(&g4DD, y_out);
    """
    
    # Register the C function with nrpy.
    # It needs the include_CodeParameters_h flag because it uses M_scale.
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body,
        include_CodeParameters_h=True
    )
    print("    ... initial_data() registration complete.")

<a id='gsl_wrapper'></a>
### 5.b: The GSL Wrapper Function

The GNU Scientific Library (GSL) provides powerful, general-purpose routines for solving systems of Ordinary Differential Equations (ODEs). To use them, we must provide a C function that calculates the RHS of our ODE system and conforms to a specific function pointer signature required by the library. The `ode_gsl_wrapper` C function serves as this critical **adapter** or **bridge** between the generic GSL interface and our specialized project code.

This Python function registers the C function that performs these steps in order every time the GSL solver takes a time step:

1.  **Unpack Parameters**: It receives a generic `void *params` pointer from the GSL solver. Its first action is to cast this pointer back to its true type, `gsl_params *`, which is our custom "carrier" struct. This gives the function access to the `commondata`, `params`, and `metric` structs needed by our physics routines.
2.  **Prepare `connection_struct`**: It declares an empty `connection_struct` on the stack. This will serve as a temporary container for the Christoffel symbols.
3.  **Call `connections()` Dispatcher**: It calls our high-level `connections()` dispatcher, passing it the necessary pointers. The dispatcher then calls the appropriate worker (e.g., `con_schwarzschild`) to compute the Christoffel symbols and fill the `conn` struct.
4.  **Call `calculate_ode_rhs()` Engine**: It then passes the current state vector `y` and the now-filled `conn` struct to our RHS engine. This engine computes the derivatives $\frac{dx^{\alpha}}{d\lambda}$ and $\frac{dp^{\alpha}}{d\lambda}$ and stores them in the output array.
5.  **Return Success**: Finally, it returns `GSL_SUCCESS`, signaling to the GSL solver that the RHS calculation was completed correctly.

In [16]:
def ode_gsl_wrapper():
    """
    Generates and registers the ode_gsl_wrapper C function. This acts as
    a bridge between the generic GSL solver and our project-specific functions.
    """
    print(" -> Generating GSL wrapper function: ode_gsl_wrapper...")


    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "gsl/gsl_errno.h"]
    desc = r"""@brief Acts as an adapter between the generic GSL ODE solver and our specific C functions.
 
    The GNU Scientific Library (GSL) ODE solver requires a function pointer to a
    function with a very specific signature. This wrapper function is designed to
    exactly match that required signature, acting as a "bridge" to our modular,
    project-specific C code.
    
    At each step of the integration, the GSL solver calls this function to
    orchestrate the calls to our physics "engine" functions in the correct order.
    
    @param[in]  lambda The current value of the independent variable (affine parameter). Unused.
    @param[in]  y      The current 8-component state vector [t, r, th, ph, p^t, ...].
    @param[in]  params A generic `void` pointer to a gsl_params struct, provided by the GSL system.
    @param[out] f      A pointer to the 8-component output array where the RHS results are stored.
    
    @return An integer status code for the GSL library (`GSL_SUCCESS` on success)."""
    name = "ode_gsl_wrapper"
    cfunc_type = "int"

    params = "double lambda, const double y[8], double f[8], void *params"


    body = r"""
    // The GSL solver doesn't use lambda, so we cast it to void to prevent compiler warnings.
    (void)lambda;

    // --- Step 1: Unpack the Carrier Struct ---
    // Cast the generic void* params pointer back to its true type, our gsl_params carrier.
    gsl_params *gsl_parameters = (gsl_params *)params;

    // --- Step 2: Prepare for Physics Calculation ---
    // Create an empty container for the Christoffel symbols on the stack.
    connection_struct conn;

    // --- Step 3: Call Engine Functions ---
    // Call the connections dispatcher to fill the conn struct.
    // We pass it the pointers to the params and metric structs that we unpacked from the carrier.
    connections(gsl_parameters->commondata, gsl_parameters->params, gsl_parameters->metric, y, &conn);

    // Call the RHS engine to compute the derivatives.
    // We pass it the necessary pointers and the output array 'f'.
    calculate_ode_rhs( y, &conn, f);

    // --- Step 4: Return Success ---
    // Return a success code to the GSL solver.
    return GSL_SUCCESS;
    """

    # Register the C function with nrpy.
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        cfunc_type=cfunc_type,
        name=name,
        params=params,
        body=body
    )
    print("    ... ode_gsl_wrapper() registration complete.")

<a id='integration_loop'></a>
### 5.c: The Main Integration Loop

This Python function registers the `integrate_single_photon()` C function. This is another high-level **orchestrator** that brings together all the components to solve the ODE system. It sets up, executes, and tears down the GSL numerical integration environment for a single photon trajectory.

The generated C code body performs several distinct tasks in sequence:

1.  **GSL Solver Setup**: It initializes the three core components required by the GSL ODE solver:
    *   **`gsl_odeiv2_step`**: The "stepper," which implements the specific mathematical algorithm for advancing the solution. We use `gsl_odeiv2_step_rkf45`, a reliable adaptive-step method known as the Runge-Kutta-Fehlberg (4,5) method.
    *   **`gsl_odeiv2_control`**: The "controller," which monitors the estimated error from the stepper at each step and adjusts the step-size to maintain a pre-defined accuracy tolerance.
    *   **`gsl_odeiv2_evolve`**: The "evolver," which combines the stepper and controller into a single object that manages the state of the solver between steps.

2.  **Define the ODE System**: The GSL library needs to be told what equations to solve and what parameters to use. This is done by setting up two structs:
    *   A `gsl_params` "carrier" struct is created to hold pointers to all the data our RHS function needs (the `commondata` struct, `params` struct, and the `metric_params` struct).
    *   A `gsl_odeiv2_system` struct is then initialized. This crucial struct bundles our derivative function (`ode_gsl_wrapper`), the number of equations (`8`), and a `void*` pointer to our `gsl_params` carrier struct.

3.  **Main Integration Loop**: The function enters a `while` loop that continues until a termination condition is met. Inside the loop, the line `gsl_odeiv2_evolve_apply(evol, control, step, ...)` is the call that tells GSL to compute the derivatives (by calling our wrapper) and advance the state vector `y` by one time-step (of variable size `d_lambda`).

4.  **Store Results**: After each successful step, it stores the photon's new affine parameter (`lambda`) and position coordinates (`y[0]` through `y[3]`) into the appropriate arrays within the `path_struct`.

5.  **Check Termination Conditions**: After storing the data, it checks for events that should stop the integration. This function needs `M_scale` for the event horizon check, so it uses the `include_CodeParameters_h` flag to make the `M_scale` variable available.
    *   The photon's radial coordinate `y[1]` has fallen below the event horizon radius ($2M$).
    *   The photon has escaped the simulation region by exceeding `r_max` or `t_max`.
    *   The GSL solver itself reports a failure or the maximum number of steps is reached.

6.  **Cleanup**: After the loop finishes, it calls the appropriate `gsl_*_free()` functions to release all memory allocated by the GSL components, preventing memory leaks.

In [17]:
def integrate_single_photon():

    print(" -> Generating main integration loop: integrate_single_photon...")


    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "gsl/gsl_errno.h", "gsl/gsl_odeiv2.h"]
    desc = r"""@brief Integrates the 8 geodesic ODEs using an adaptive GSL solver.
 
    This function is a high-level "orchestrator" that brings together all the
    components to solve the geodesic ODE system. It initializes, executes, and
    tears down the GNU Scientific Library (GSL) numerical integration environment.
    
    The core of this function is a while loop that repeatedly calls the GSL
    evolver to advance the photon's state by one adaptive step. After each
    step, it records the new position and checks for various termination
    conditions, such as hitting the event horizon or escaping the simulation domain.
    
    @param[in] params             A pointer to the main params_struct containing all configurable parameters.
    @param[in] metric             A pointer to the metric_params struct specifying the metric to use.
    @param[in] initial_lambda     The starting value of the affine parameter, lambda.
    @param[in] d_lambda_initial   The initial step-size guess for the adaptive GSL solver.
    @param[in] num_steps          The maximum number of integration steps to take before stopping.
    @param[in] start_y            The initial 8-component state vector [t, r, th, ph, p^t, p^r, p^th, p^ph].
    @param[in] r_max              The maximum radial coordinate before the integration is terminated.
    @param[in] t_max              The maximum time coordinate before the integration is terminated.
    @param[out] path_out          A pointer to the path_struct that will be filled with the full trajectory data."""
        
    name = "integrate_single_photon"


    params = """const commondata_struct *restrict commondata,
    const params_struct *restrict params,
    const metric_params *restrict metric,
    const double initial_lambda,
    const double d_lambda_initial,
    const int num_steps,
    const double start_y[8],
    const double r_max,
    const double t_max,
    path_struct *restrict path_out"""

    body = r"""
    // --- GSL Solver Setup ---


    const gsl_odeiv2_step_type * T = gsl_odeiv2_step_rkf45;
    gsl_odeiv2_step * step = gsl_odeiv2_step_alloc(T, 8);
    gsl_odeiv2_control * control = gsl_odeiv2_control_y_new(1e-9, 1e-9);
    gsl_odeiv2_evolve * evol = gsl_odeiv2_evolve_alloc(8);

    // --- THIS IS THE KEY CHANGE ---
    // 1. Create the gsl_params carrier struct.
    gsl_params gsl_parameters;
    // 2. Populate it with pointers to the actual data structs.
    gsl_parameters.metric = metric;
    gsl_parameters.commondata = commondata;

    // 3. Pass a pointer to this carrier struct as the final argument to gsl_odeiv2_system.
    gsl_odeiv2_system sys = {ode_gsl_wrapper, NULL, 8, &gsl_parameters};

    // --- Initial Conditions & Main Loop ---
    double lambda = initial_lambda;
    double d_lambda = d_lambda_initial;
    double y[8];
    for (int j = 0; j < 8; j++) { y[j] = start_y[j]; }

    path_out->lambda[0] = lambda;
    path_out->t[0] = y[0];
    path_out->r[0] = y[1];
    path_out->theta[0] = y[2];
    path_out->phi[0] = y[3];
    path_out->actual_steps = 0;
    path_out->reason = IN_FLIGHT;

    int i = 0;
    while (i < num_steps) {
        int status = gsl_odeiv2_evolve_apply(evol, control, step, &sys, &lambda, 1e10, &d_lambda, y);
        if (status != GSL_SUCCESS) {
        path_out->reason = MAX_STEPS_REACHED;
        break;
        }
        i++;

        path_out->actual_steps = i;
        path_out->lambda[i] = lambda;
        path_out->t[i] = y[0];
        path_out->r[i] = y[1];
        path_out->theta[i] = y[2];
        path_out->phi[i] = y[3];

        if (y[1] <= 2.0 *M_scale) {
        path_out->reason = HIT_HORIZON;
        break;
        }
        if (y[1] > r_max || y[0] > t_max) {
        path_out->reason = (y[1] > r_max) ? ESCAPED_R_MAX : ESCAPED_T_MAX;
        break;
        }
    }

    // --- GSL Memory Cleanup ---
    gsl_odeiv2_evolve_free(evol);
    gsl_odeiv2_control_free(control);
    gsl_odeiv2_step_free(step);
    """

    # Register the C function. Note that include_CodeParameters_h is removed.
    cfc.register_CFunction(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body,
        include_CodeParameters_h=True
    )
    print("    ... integrate_single_photon() registration complete.")

<a id='main_entry_point'></a>
### 5.d: The `main()` C Function Entry Point

This function registers the C `main()` function, which serves as the entry point for the entire executable program. The C code for this function is handwritten as a string. In our final architecture, `main()` is a pure **orchestrator**; it contains no physics logic itself. Instead, it calls other functions to set up the simulation, run the integration, and handle the output.

The `main` function performs the full sequence of operations required for a run:

1.  **Declare Data Structures**: It declares instances of all the necessary structs (`commondata_struct`, `params_struct`, `metric_params`) and the `start_y` state vector array.
2.  **Initialize Parameters**: It initializes the parameters in a two-step process that makes the code robust and flexible:
    *   First, it calls `commondata_struct_set_to_default()` to populate the `commondata` struct with the default values that were compiled into the executable (e.g., `M_scale = 1.0`).
    *   Next, it calls `cmdline_input_and_parfile_parser()`. This function reads the project's `.par` file and the command line, and it will **override** the compiled-in defaults with any values provided by the user at runtime.
3.  **Initialize State Vector**: It makes a single call to our high-level `initial_data()` orchestrator. This function handles all the complex logic of setting the initial position, momentum, and computing $p^0$.
4.  **Allocate Memory**: It allocates memory for the arrays inside the `path_struct` using `malloc()`. This struct will be used to store the full trajectory (the output) of the integration.
5.  **Run Simulation**: It calls the top-level `integrate_single_photon()` function, passing it all the necessary pointers and simulation bounds. This call initiates the main GSL integration loop.
6.  **Write Output**: After the integration is complete, it opens a text file (`photon_path.txt`) and uses `fprintf` to write the stored trajectory data from the `path_struct` into a human-readable, tabulated format.
7.  **Cleanup**: In the final step, it calls `free()` to release all the memory that was allocated for the `path_struct` arrays, preventing any memory leaks.

In [18]:

def main():
   
    print(" -> Generating C entry point: main()...")
    
    includes = ["BHaH_defines.h", "BHaH_function_prototypes.h", "stdio.h", "stdlib.h"]
    desc = r"""@brief Main entry point for the photon geodesic integration program.
    This function orchestrates a complete, single simulation run. It is responsible
    for setting up all necessary data structures, invoking the initial data routine,
    running the main integration loop, and handling the final output of the results."""
    
    cfunc_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"

    # This body is now a pure high-level orchestrator.
    body = r"""
    (void)argc; (void)argv;

    // Step 1: Declare all necessary structs and the state vector array.
    commondata_struct commondata = {0};
    params_struct params = {0}; // Still needed for function signatures
    metric_params metric = {0};
    double start_y[8];

    // Step 2: Initialize the parameter structs with default values.
    commondata_struct_set_to_default(&commondata);

    cmdline_input_and_parfile_parser(&commondata, argc, argv);
    // params_struct_set_to_default() would be called here if needed.
    metric.type = Schwarzschild;
    
    // Step 3: Set the complete initial state vector by calling the orchestrator.
    initial_data(&commondata, &params, &metric, start_y);

    // Step 4: Allocate memory for storing the photon's path.
    path_struct path_results;
    const int num_steps = 80000;
    path_results.num_steps = num_steps;
    path_results.lambda = (double*)malloc(sizeof(double) * (num_steps + 1));
    path_results.t      = (double*)malloc(sizeof(double) * (num_steps + 1));
    path_results.r      = (double*)malloc(sizeof(double) * (num_steps + 1));
    path_results.theta  = (double*)malloc(sizeof(double) * (num_steps + 1));
    path_results.phi    = (double*)malloc(sizeof(double) * (num_steps + 1));

    // Step 5: Run the simulation.
    printf("Starting photon integration...\n");
    const double r_max = 50.0;
    const double t_max = 200.0;
    const double initial_lambda = 0.0;
    const double d_lambda_initial = 0.01;

    integrate_single_photon(&commondata, &params, &metric, initial_lambda, d_lambda_initial, 
                            num_steps, start_y, r_max, t_max, &path_results);

    printf("Integration finished after %d steps. Reason code: %d\n", path_results.actual_steps, path_results.reason);

    // Step 6: Write the results to a file.
    FILE *fp = fopen("photon_path.txt", "w");
    if (fp == NULL) { return 1; }
    fprintf(fp, "# lambda\tt\tr\ttheta\tphi\n");
    for (int i = 0; i <= path_results.actual_steps; i++) {
        fprintf(fp, "%.5f\t%.5f\t%.5f\t%.5f\t%.5f\n",
            path_results.lambda[i], path_results.t[i], path_results.r[i],path_results.theta[i],
            path_results.phi[i]);
    }
    fclose(fp);
    printf("Path data written to photon_path.txt\n");

    // Step 7: Clean up allocated memory.
    free(path_results.lambda); free(path_results.t); free(path_results.r);
    free(path_results.theta); free(path_results.phi);
    
    return 0;
    """
    
    cfc.register_CFunction(
        includes=includes, 
        desc=desc, 
        cfunc_type=cfunc_type,
        name=name, 
        params=params, 
        body=body
    )
    print("    ... main() registration complete.")

<a id='assemble_project'></a>
# Step 6: Project Assembly

The final phase of the notebook brings all the previously defined pieces together to construct the complete, compilable C project.

<a id='register_structs'></a>
### 6.a: Custom Data Structures

This function defines all the necessary C `struct` and `enum` types for the project. It then registers them with the BHaH infrastructure, which makes these custom data types available to all other C files via the master header `BHaH_defines.h`.

The function `register_custom_structures_and_params` performs the following actions:
1.  **Generates `connection_struct`**: It programmatically creates the C `typedef` for the `connection_struct`. This struct contains 40 `double` members to hold the unique Christoffel symbols ($\Gamma^{\alpha}_{\mu\nu}$). This is done by looping and generating a list of C variable names.
2.  **Generates `metric_struct`**: It follows the same programmatic pattern to create the `metric_struct`, which contains 10 `double` members to hold the unique components of the metric tensor ($g_{\mu\nu}$).
3.  **Defines Other Structs**: It defines the C `typedef`s for all other data structures (`Metric_t` enum, `metric_params`, the GSL "carrier" struct `gsl_params`, and the `path_struct` for storing results) as Python strings.
4.  **Registers with `BHaH`**: For each `struct` or `enum`, it calls `Bdefines_h.register_BHaH_defines()`. This function, part of the `nrpy.infrastructures.BHaH.BHaH_defines_h` module, adds the C code string to a global registry. When `Bdefines_h.output_BHaH_defines_h()` is called later in the build process, it will automatically find and include all these registered definitions in the final header file.

In [19]:
def register_custom_structures_and_params():
    """
    Generates C code for all custom structs and enums, then registers them with BHaH.
    """
    print("Registering custom C data structures...")

    # --- 1. Register the connections_struct ---
    list_of_connections = []
    Gamma4UDD = ixp.declarerank3("Gamma4UDD", dimension=4)
    for i in range(4):
        for j in range(4):
            for k in range(j, 4):
                list_of_connections.append(str(Gamma4UDD[i][j][k]))
    # FIX #1: The joiner is now a semicolon, not a comma.
    struct_members_string = "double " + ";\n        double ".join(list_of_connections) + ";"
    connections_struct_str = f"typedef struct {{\n        {struct_members_string}\n}} connection_struct;"
    Bdefines_h.register_BHaH_defines("connections_struct", connections_struct_str)
    print(" -> Registered C struct: connections_struct")

    # --- 2. Register the metric and GSL parameter structs ---

    list_of_metric_components = []
    for nu in range(4):
        for mu in range(nu,4):
            list_of_metric_components.append(f"g{nu}{mu}")
    struct_members_string = "double " + ";\n        double ".join(list_of_metric_components) + ";"

    metric_struct_str = f"""
    typedef struct {{
            {struct_members_string}
    }} metric_struct;
    """
    Bdefines_h.register_BHaH_defines("metric_struct", metric_struct_str)
    
    # FIX #2: metric_params and gsl_params are now registered together to guarantee order.
    metric_type_enum_str = "typedef enum { Schwarzschild, Kerr, Numerical } Metric_t;"
    metric_params_struct_str = "typedef struct { Metric_t type; } metric_params;"
    # The gsl_params struct depends on both metric_params and the auto-generated params_struct.
    gsl_params_struct_str = "typedef struct { const commondata_struct *commondata; const params_struct *params; const metric_params *metric; } gsl_params;"
    # Register them all in one block.
    Bdefines_h.register_BHaH_defines("metric_and_gsl_params", f"{metric_type_enum_str}\n{metric_params_struct_str}\n{gsl_params_struct_str}")
    print(" -> Registered C structs: Metric_t, metric_params, gsl_params")

    # --- 3. Register the path_struct and its enum ---
    # This was already correct, but we keep it for consistency.
    termination_reason_enum_str = "typedef enum { IN_FLIGHT, ESCAPED_R_MAX, ESCAPED_T_MAX, HIT_HORIZON, MAX_STEPS_REACHED } termination_reason_t;"
    photon_path_struct_str = "typedef struct { int num_steps; int actual_steps; termination_reason_t reason; double *lambda, *t, *r, *theta, *phi; } path_struct;"
    Bdefines_h.register_BHaH_defines("path_struct", f"{termination_reason_enum_str}\n{photon_path_struct_str}")
    print(" -> Registered C structs: termination_reason_t, path_struct")

<a id='final_build'></a>
### 6.b: Final Build and Compilation

This is the main execution block of the notebook. It brings all the previously defined Python functions together and calls them in a precise sequence to generate every file needed for the final, compilable C project.

The sequence of operations is critical, as later steps depend on the files and registrations created by earlier ones:

1.  **Register All Components**: It calls all the C-generating Python functions that we have defined throughout the notebook (`register_custom_structures_and_params`, `g4DD_schwarzschild`, `initial_data`, etc.). This populates `nrpy`'s internal library with the complete definitions for all our custom C data structures and functions. At this stage, no files have been written yet; everything exists only in memory.

2.  **Generate Parameter Handling Files**: It calls the necessary functions from the BHaH infrastructure to set up the parameter system:
    *   `CPs.write_CodeParameters_h_files()`: Generates `CodeParameters.h` (which defines the `commondata_struct` and `params_struct` types) and `set_CodeParameters.h` (which contains the C code for unpacking parameters into local variables).
    *   `CPs.register_CFunctions_params_commondata_struct_set_to_default()`: Registers the C functions that initialize the parameter structs with their compiled-in default values.
    *   `cmdline_input_and_parfiles.generate_default_parfile()`: Creates the `project_name.par` file.
    *   `cmdline_input_and_parfiles.register_CFunction_cmdline_input_and_parfile_parser()`: Registers the C function that reads the `.par` file at runtime.

3.  **Generate `BHaH_defines.h`**: It calls `Bdefines_h.output_BHaH_defines_h()`. This function scans `nrpy`'s internal library for all registered data structures (like `metric_struct` and `connection_struct`) and writes them into the master C header file, `BHaH_defines.h`.

4.  **Copy Helper Files**: It calls `gh.copy_files()` to copy any necessary dependency files from the `nrpy` library installation into our project directory.

5.  **Generate C Source, Prototypes, and Makefile**: It calls the final, most important build function, `Makefile.output_CFunctions_function_prototypes_and_construct_Makefile()`. This powerful function from `nrpy/infrastructures/BHaH/Makefile_helpers.py` performs three tasks at once:
    *   It iterates through every C function registered with `nrpy.c_function.register_CFunction` and writes each one into its own `.c` file (e.g., `main.c`, `connections.c`).
    *   It generates `BHaH_function_prototypes.h`, a header file containing the function declarations (prototypes) for all the generated `.c` files. This is crucial as it allows the different C files to call functions defined in one another.
    *   It constructs the `Makefile`, which contains the compilation and linking instructions needed to build the final executable program. It is also configured to automatically link against the required GSL libraries.

After this cell is run, a complete, self-contained, and ready-to-compile C project will exist in the output directory.

In [20]:
# In your final build cell (id: fc265b8f)

import nrpy.infrastructures.BHaH.CodeParameters as CPs

print("\\nAssembling and building C project...")

# Create the output directory if it doesn't exist.
os.makedirs(project_dir, exist_ok=True)

# --- Call all registration functions for user-defined C code ---
# This populates nrpy's in-memory list of C functions and data structures.
register_custom_structures_and_params()

# Register metric and connection engines and dispatchers
g4DD_schwarzschild()
g4DD_metric()
con_schwarzschild()
connections()

# Register ODE and initial data engines
calculate_ode_rhs()
calculate_p0()
initial_data()

# Register high-level orchestrators
ode_gsl_wrapper()
integrate_single_photon()
main()


# --- Call BHaH infrastructure functions to generate parameter handling ---
# 1. Generates CodeParameters.h, which defines the structs
CPs.write_CodeParameters_h_files(project_dir=project_dir)
# 2. Generates the C functions to initialize the structs with default values
CPs.register_CFunctions_params_commondata_struct_set_to_default()

cmdline_input_and_parfiles.generate_default_parfile(
    project_dir=project_dir, project_name=project_name
)
# 4. Generates the C function that parses the .par file at runtime
cmdline_input_and_parfiles.register_CFunction_cmdline_input_and_parfile_parser(
    project_name=project_name
)

# --- Generate the core BHaH header file ---
# This file includes all registered structs, enums, and macro definitions.
print("\\nGenerating BHaH master header file...")
Bdefines_h.output_BHaH_defines_h(project_dir=project_dir)

# --- Copy any necessary helper files from the nrpy library ---
print("Copying required helper files...")
gh.copy_files(
    package="nrpy.helpers",
    filenames_list=["simd_intrinsics.h"],
    project_dir=project_dir,
    subdirectory="simd",
)

# --- Generate all C source files, function prototypes, and the Makefile ---
# This is the final step that writes all registered C code to disk and creates the build system.
print("Generating C source files, prototypes, and Makefile...")
addl_CFLAGS = ["-Wall -Wextra -g $(shell gsl-config --cflags)"]
Makefile.output_CFunctions_function_prototypes_and_construct_Makefile(
    project_dir=project_dir,
    project_name=project_name,
    exec_or_library_name=project_name,
    addl_CFLAGS=addl_CFLAGS,
    addl_libraries=["$(shell gsl-config --libs)"],
)

print(f"\\nFinished! A C project has been generated in {project_dir}/")
print(f"To build, navigate to this directory in your terminal and type 'make'.")
print(f"To run, type './{project_name}'.")

\nAssembling and building C project...
Registering custom C data structures...
 -> Registered C struct: connections_struct
 -> Registered C structs: Metric_t, metric_params, gsl_params
 -> Registered C structs: termination_reason_t, path_struct
 -> Generating C worker function: g4DD_schwarzschild()...
    ... g4DD_schwarzschild() registration complete.
 -> Generating C dispatcher function: g4DD_metric()...
    ... g4DD_metric() registration complete.
 -> Generating C dispatcher: connections()...
    ... connections() registration complete.
 -> Generating C engine function: calculate_p0()...
    ... calculate_p0() registration complete.
 -> Generating C function: initial_data()...
    ... initial_data() registration complete.
 -> Generating GSL wrapper function: ode_gsl_wrapper...
    ... ode_gsl_wrapper() registration complete.
 -> Generating main integration loop: integrate_single_photon...
    ... integrate_single_photon() registration complete.
 -> Generating C entry point: main()..